# Analyzing Arthropoda Taxonomy: Integrating GBIF, NCBI and iNaturalist Data

![title](https://wallpapercave.com/wp/wp1870417.jpg)

This Python notebook is designed for the purpose of integrating taxonomic data from two major biological databases, GBIF (Global Biodiversity Information Facility) and NCBI (National Center for Biotechnology Information) and iNaturalist, to enhance the accuracy and comprehensiveness of ecological and biological research. GBIF primarily focuses on biodiversity data including species distribution and ecological information, whereas NCBI provides a broader range of data including genomic and taxonomic details. 

Combining these sources enables researchers to cross-validate species identifications and improve the richness of ecological datasets with genetic information. A key biological task performed in this notebook is the construction of a taxonomic tree, which helps in visualizing and understanding the evolutionary relationships and classification hierarchy among different species within a chosen taxon (in this case, the Arthropda pyhlum).

## 1. Importing libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
import taxonmatch as txm

## 2. Downloading and processing samples

The initial steps involve downloading the most recent taxonomic data from GBIF and NCBI to ensure the analysis is based on the latest available information. 

In [ ]:
gbif_dataset = txm.download_gbif_taxonomy()

In [ ]:
ncbi_dataset = txm.download_ncbi_taxonomy()

## 2.1 Checking Inconsistencies in nomenclature

Matching based on canonical names between the GBIF and NCBI datasets is unreliable due to significant taxonomic inconsistencies. In particular, the same canonical name may be assigned to multiple kingdoms, highlighting classification discrepancies. Even when the taxonomic status is accepted, the taxonomic structures in GBIF and NCBI can differ substantially. This necessitates filtering and evaluating differences before considering a match valid, preventing false correspondences between incongruent taxonomies.

In [ ]:
df_inconsistencies = txm.get_inconsistencies(gbif_dataset, ncbi_dataset)

In [ ]:
df_inconsistencies.sample(5)

## 3.a Training the classifier from scratch

If required, the notebook outlines steps to train a machine learning classifier to distinguish between correct and incorrect taxonomic matches. This involves generating positive and negative examples, preparing the training dataset, and comparing different models.

In [ ]:
#positive_matches = txm.generate_positive_set(gbif_dataset, ncbi_dataset, 25000)

In [ ]:
#negative_matches = txm.generate_negative_set(gbif_dataset, ncbi_dataset, 25000)

In [ ]:
#full_training_set = txm.prepare_data(positive_matches, negative_matches)

In [ ]:
#full_training_set.to_csv("training_set.txt", index = False)

In [ ]:
#X_train, X_test, y_train, y_test = txm.generate_training_test(full_training_set)

In [ ]:
#model = txm.XGBClassifier(learning_rate=0.1,n_estimators=500, max_depth=9, n_jobs=-1, colsample_bytree = 1, subsample = 0.8)

In [ ]:
#model.fit(X_train, y_train, verbose=False)

In [ ]:
#model.save_model("xgb_model.json")

## 3.b Compare and load pre-trained models

 Alternatively, it provides the option to load a pre-trained model, simplifying the process for routine analyses.

In [ ]:
model = txm.load_xgb_model() 

The following steps explain why XGBoost was selected as the default model in TaxonMatch.

In [ ]:
full_training_set = txm.load_training_set()

In [ ]:
X_train, X_test, y_train, y_test = txm.generate_training_test(full_training_set)

In [ ]:
df_results, trained_models = txm.compare_models(X_train, X_test, y_train, y_test, cv_folds=5, random_seed=42)

In [ ]:
df_results

In [ ]:
txm.plot_roc_curves(trained_models, X_test, y_test, top_n=10, save_path="./roc_top10.png")

## 4. Match NCBI with GBIF dataset 

In this section, the focus is on comparing and aligning the taxonomic data from NCBI and GBIF datasets. It specifically targets the taxon "Arthropoda" to narrow down the analysis of this clade. Using a pre-trained machine learning model, the notebook matches records from both datasets, categorizing them as exact matches, unmatched, or potentially mislabeled due to typographical errors

In [ ]:
gbif_arthropoda, ncbi_arthropoda = txm.select_taxonomic_clade("arthropoda", gbif_dataset, ncbi_dataset)

In [ ]:
matched_df, unmatched_df, possible_typos_df = txm.match_dataset(gbif_arthropoda, ncbi_arthropoda, model, tree_generation = True)

## 5. Generate the taxonomic tree 

In the last section, the notebook constructs a taxonomic tree from the matched and unmatched data between the GBIF and NCBI datasets, focusing on the Arthropoda phylum. This visual representation helps to illustrate the evolutionary relationships and classification hierarchy among the species. The tree is then converted into a dataframe for further analysis and saved in textual format for documentation and review purposes.

In [ ]:
tree = txm.generate_taxonomic_tree(matched_df, unmatched_df)

In [ ]:
#txm.print_tree(tree)

In [ ]:
cicadetta_tree = txm.reroot_tree(tree, root_name="cicadetta")

In [ ]:
txm.print_tree(cicadetta_tree)

In [ ]:
df_from_tree = txm.convert_tree_to_dataframe(tree, gbif_dataset[1], ncbi_dataset[1], "df_arthropoda.txt", index=False)

In [ ]:
txm.save_tree(tree, "./arthropoda_tree.txt", output_format='txt')

## 6. Add iNaturalist information

In [ ]:
inat_dataset = txm.download_inat_taxonomy()

In [ ]:
inat_tree = txm.add_inat_taxonomy(tree, inat_dataset)

In [ ]:
cicadetta_tree_with_inat = txm.reroot_tree(inat_tree, root_name="cicadetta")

In [ ]:
txm.print_tree(cicadetta_tree_with_inat)

In [ ]:
df_with_inat = txm.convert_tree_to_dataframe(inat_tree, gbif_dataset[1], ncbi_dataset[1], "df_arthropoda_with_inat.txt", inat_dataset=inat_dataset, index=True)

In [ ]:
txm.save_tree(inat_tree, "arthropoda_tree_with_inat.txt", output_format='txt')